In [1]:
# -- driver_run_forecast_LV1.py  --
# master python script to do a full LV1 forecast simulation


import sys
#import pickle
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime, timezone
#import xarray as xr
#import netCDF4 as nc

##############


In [2]:
sys.path.append('../sdpm_py_util')

import atm_functions as atmfuns
import ocn_functions as ocnfuns
import grid_functions as grdfuns
import util_functions as utlfuns 
import plotting_functions as pltfuns
from util_functions import s_coordinate_4
from get_PFM_info import get_PFM_info
from make_LV1_dotin_and_SLURM import make_LV1_dotin_and_SLURM
from run_slurm_LV1 import run_slurm_LV1



/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/ocn_functions.py:15: UserWarning: The seawater library is deprecated! Please use gsw instead.
  import seawater


In [3]:
PFM=get_PFM_info()

run_type = 'forecast'
# we will use hycom for IC and BC
ocn_mod = PFM['ocn_model']
print('ocean boundary and initial conditions will be from:')
print(ocn_mod)
# we will use nam_nest for the atm forcing
atm_mod = PFM['atm_model']
print('atm forcing will be from:')
print(atm_mod)
# we will use opendap, and netcdf to grab ocn, and atm data
get_method = 'open_dap_nc'

get_PFM_info(): running onswell
ocean boundary and initial conditions will be from:
hycom
atm forcing will be from:
nam_nest


In [4]:
# figure out what the time is local and UTC
start_time = datetime.now()
utc_time = datetime.now(timezone.utc)
year_utc = utc_time.year
month_utc = utc_time.month
day_utc = utc_time.day
hour_utc = utc_time.hour


In [5]:
print("Starting: driver_run_forecast_LV1: Current local Time =", start_time, "UTC = ",utc_time)

if hour_utc < 12:
    hour_utc=12
    day_utc=day_utc-1  # this only works if day_utc \neq 1

yyyymmdd = "%d%02d%02d" % (year_utc, month_utc, day_utc)
    
#yyyymmdd = '20240717'
# the hour in Z of the forecast, hycom has forecasts once per day starting at 1200Z
hhmm='1200'
forecastZdatestr = yyyymmdd+hhmm+'Z'   # this could be used for model output to indicate when model was initialized.

yyyymmdd = '20240728'
print("Preparing forecast starting on ",yyyymmdd)



Starting: driver_run_forecast_LV1: Current local Time = 2024-07-30 14:08:17.757216 UTC =  2024-07-30 21:08:17.757240+00:00
Preparing forecast starting on  20240728


In [6]:
# get the ROMS grid as a dict
RMG = grdfuns.roms_grid_to_dict(PFM['lv1_grid_file'])



In [7]:
# make a switch to see if this file exists. If it exists, we don't need to run the code in this block
# first the atm data
# get the data as a dict
# need to specify hhmm because nam forecast are produced at 6 hr increments
ATM = atmfuns.get_atm_data_as_dict(yyyymmdd,hhmm,run_type,atm_mod,'open_dap_nc',PFM)
# put in a function to check to make sure that all the data is good
# put in a function to plot the raw atm data if we want to



getting atm forecast for:
[datetime.datetime(2024, 7, 28, 12, 0)
 datetime.datetime(2024, 7, 28, 15, 0)
 datetime.datetime(2024, 7, 28, 18, 0)
 datetime.datetime(2024, 7, 28, 21, 0)
 datetime.datetime(2024, 7, 29, 0, 0) datetime.datetime(2024, 7, 29, 3, 0)
 datetime.datetime(2024, 7, 29, 6, 0) datetime.datetime(2024, 7, 29, 9, 0)
 datetime.datetime(2024, 7, 29, 12, 0)
 datetime.datetime(2024, 7, 29, 15, 0)
 datetime.datetime(2024, 7, 29, 18, 0)
 datetime.datetime(2024, 7, 29, 21, 0)
 datetime.datetime(2024, 7, 30, 0, 0) datetime.datetime(2024, 7, 30, 3, 0)
 datetime.datetime(2024, 7, 30, 6, 0) datetime.datetime(2024, 7, 30, 9, 0)
 datetime.datetime(2024, 7, 30, 12, 0)
 datetime.datetime(2024, 7, 30, 15, 0)
 datetime.datetime(2024, 7, 30, 18, 0)
 datetime.datetime(2024, 7, 30, 21, 0)
 datetime.datetime(2024, 7, 31, 0, 0)]


In [8]:
# plot some stuff
pltfuns.plot_atm_fields(ATM, RMG, PFM)
print('done with plotting ATM fields')



/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/plotting_functions.py:72: RuntimeWarning: overflow encountered in square
  magnitude = np.sqrt(U**2 + V**2)


done with plotting ATM fields


In [9]:
# put the atm data on the roms grid, and rotate the velocities
# everything in this dict turn into the atm.nc file

ATM_R  = atmfuns.get_atm_data_on_roms_grid(ATM,RMG)
print('done with: atmfuns.get_atm_data_on_roms_grid(ATM,RMG)')
# all the fields plotted with the data on roms grid


done with: atmfuns.get_atm_data_on_roms_grid(ATM,RMG)


In [10]:

pltfuns.plot_all_fields_in_one(ATM, ATM_R, RMG, PFM)
print('done with: pltfuns.plot_all_fields_in_one(ATM, ATM_R, RMG, PFM)')


/home/mspydell/models/PFM_root/PFM/driver/../sdpm_py_util/plotting_functions.py:291: RuntimeWarning: overflow encountered in square
  magnitude = np.sqrt(U**2 + V**2)


done with: pltfuns.plot_all_fields_in_one(ATM, ATM_R, RMG, PFM)


<Figure size 640x480 with 0 Axes>

In [11]:
# output a netcdf file of ATM_R
# make the atm .nc file here.
# fn_out is the name of the atm.nc file used by roms
fn_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_atm_file'] # LV1 atm forcing filename
print('driver_run_forcast_LV1: saving ATM file to ' + fn_out)
atmfuns.atm_roms_dict_to_netcdf(ATM_R,fn_out)
print('driver_run_forecast_LV1:  done with writing ATM file, Current time ', datetime.now())
# put in a function to plot the atm.nc file if we want to
pltfuns.load_and_plot_atm(RMG, PFM)
print('done with pltfuns.load_and_plot_atm(PFM)')



driver_run_forcast_LV1: saving ATM file to /scratch/PFM_Simulations/LV1_Forecast/Forc/LV1_ATM_FORCING.nc
<xarray.Dataset>
Dimensions:     (tair_time: 21, er: 390, xr: 253, pair_time: 21, qair_time: 21,
                 wind_time: 21, rain_time: 21, srf_time: 21, lrf_time: 21,
                 time: 21)
Coordinates:
    lat         (er, xr) float64 28.52 28.53 28.54 28.55 ... 36.38 36.39 36.39
    lon         (er, xr) float64 -120.3 -120.2 -120.2 ... -118.8 -118.8 -118.8
    ocean_time  (time) float64 9.34e+03 9.341e+03 ... 9.343e+03 9.343e+03
  * tair_time   (tair_time) float64 9.34e+03 9.341e+03 ... 9.343e+03 9.343e+03
  * pair_time   (pair_time) float64 9.34e+03 9.341e+03 ... 9.343e+03 9.343e+03
  * qair_time   (qair_time) float64 9.34e+03 9.341e+03 ... 9.343e+03 9.343e+03
  * wind_time   (wind_time) float64 9.34e+03 9.341e+03 ... 9.343e+03 9.343e+03
  * rain_time   (rain_time) float64 9.34e+03 9.341e+03 ... 9.343e+03 9.343e+03
  * srf_time    (srf_time) float64 9.34e+03 9.341e+03 ..

['ncks', '-d', 'time,2024-07-28T12:00,2024-07-29T00:00', '-d', 'lon,235.0,245.0', '-d', 'lat,28.0,37.0', '-v', 'surf_el,water_temp,salinity,water_u,water_v,depth', 'https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0/FMRC/runs/GLBy0.08_930_FMRC_RUN_2024-07-28T12:00:00Z', '-4', '-O', '/scratch/PFM_Simulations/LV1_Forecast/Forc/hycom_test_mss.nc']


In [ ]:

# note, this function is hard wired to return 2.5 days of data
# also note that the first time of this data is yyyymmdd 12:00Z
# so we grab nam atm forecast data starting at this hour too.
OCN = ocnfuns.get_ocn_data_as_dict(yyyymmdd,run_type,ocn_mod,'ncks',PFM)
print('driver_run_forecast_LV1: done with get_ocn_data_as_dict: Current time ',datetime.now() )
# add OCN plotting function here !!!!

# note this takes 24.5 minutes to run on my laptop
# 3 times this timed out
# will likely need to use a wget method and directly download .nc files (arh)
# maybe downloading the netcdf file would be quicker? 

# %%
# put the ocn data on the roms grid
print('starting: ocnfuns.hycom_to_roms_latlon(OCN,RMG)')
OCN_R  = ocnfuns.hycom_to_roms_latlon(OCN,RMG)
print('driver_run_forecast_LV1: done with hycom_to_roms_latlon')
# add OCN + OCN_R plotting function here !!!

# %%
# get the OCN_IC dictionary
OCN_IC = ocnfuns.ocn_r_2_ICdict(OCN_R,RMG)
print('driver_run_forecast_LV1: done with ocn_r_2_ICdict')
# add OCN_IC.nc plotting function here !!!!
ic_file_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_ini_file']
ocnfuns.ocn_roms_IC_dict_to_netcdf(OCN_IC, icfileout)


In [ ]:
# %%
# get the OCN_BC dictionary
bc_file_out = PFM['lv1_forc_dir'] + '/' + PFM['lv1_bc_file']
OCN_BC = ocnfuns.ocn_r_2_BCdict(OCN_R,RMG)
print('driver_run_forecast_LV1: done with ocn_r_2_BCdict')


In [ ]:


# %%
ocnfuns.ocn_roms_BC_dict_to_netcdf(OCN_BC, bc_file_out)

print('driver_run_forecast_LV1:  now make .in and .sb files')

pfm_driver_src_dir = os.getcwd()
os.chdir('../sdpm_py_util')
make_LV1_dotin_and_SLURM( PFM )

# run command will be
run_slurm_LV1(PFM)

os.chdir(pfm_driver_src_dir)


# postprocess figure generation
# plot fields from his.nc



In [ ]:
# a block of code to use ncks...
import time
import subprocess

vstr = 'surf_el,water_temp,salinity,water_u,water_v,depth'

# make the ocn IC and BC .nc files here
# fn*_out are the names of the the IC.nc and BC.nc roms files
lv1_forc_dir = PFM['lv1_forc_dir']   #'/Users/mspydell/research/FF2024/models/SDPM_mss/atm_stuff/ocn_test_IC_file.nc'
full_fn_out = lv1_forc_dir + '/hycom_test_mss.nc'

west =  PFM['latlonbox'][2]+360.0
east =  PFM['latlonbox'][3]+360.0
south = PFM['latlonbox'][0]
north = PFM['latlonbox'][1]

yyyy = yyyymmdd[0:4]
mm = yyyymmdd[4:6]
dd = yyyymmdd[6:8]

# time limits
dstr0 = yyyy + '-' + mm + '-' + dd + 'T12:00'
dstr1 = yyyy + '-' + mm + '-' + str( int(dd) + 1 ) + 'T00:00'

url='https://tds.hycom.org/thredds/dodsC/GLBy0.08/expt_93.0/FMRC/runs/' 
url2 = 'GLBy0.08_930_FMRC_RUN_' + yyyy + '-' + mm + '-' + dd + 'T12:00:00Z' 
url3 = url + url2
cmd_list = ['ncks',
    '-d', 'time,'+dstr0+','+dstr1,
    '-d', 'lon,'+str(west)+','+str(east),
    '-d', 'lat,'+str(south)+','+str(north),
    '-v', vstr,
    url3 ,
    '-4', '-O', full_fn_out]

print(cmd_list)

# run ncks
tt0 = time.time()
ret1 = subprocess.call(cmd_list)


